In [1]:
path_root = r"C:/Users/jgpg000.edu/Desktop/Agente inteligente recuperación sensórica IoT/"

# ruta clases personalizadas
import sys
path_classes = f"{path_root}classes"
sys.path.append(path_classes)

## LIBRERIAS
import pandas as pd
from class_data import *
from fun_models import *

c:\Users\jgpg000.edu\AppData\Local\anaconda3\envs\management\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Cargar base de datos

In [2]:
path_excel = f"{path_root}datasets/database_items/item_list.xlsx"
path_saved_json = f"{path_root}datasets/database_items/database_json/"

documentos = load_documents(path_excel, path_saved_json)
documentos_langchain = load_documents_langchain(documentos)

from langchain_text_splitters import RecursiveCharacterTextSplitter
# Aplicamos el splitter de Langchain
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=30,
    length_function=len,
    is_separator_regex=False,
)

# Dividimos cada texto en chunks
chunks_texto = text_splitter.split_documents(documentos_langchain)
print(f"Hay {len(chunks_texto)} chunks de texto.")

100%|██████████| 88/88 [00:00<00:00, 255.53it/s]


Se han cargado 86 documentos.


100%|██████████| 86/86 [00:00<00:00, 9378.60it/s]


Se han generado 954 documentos de Langchain.
    De ellos, 86 son el texto y 868 son resúmenes de tablas.
Hay 7218 chunks de texto.


#### Configurar Agente

In [3]:
path_saved_vector = f"{path_root}datasets/database_items/database_vectorial/"
name_database = "large_esp"
modelo_llm = "gpt-4o"

template_sistema = """ Eres un asistente de una empresa encargada de smart cities.
Como preguntas, te van a adjuntar listas de requisitos de equipo para proyecto.
Tu labor es responder con el nombre de los equipos que encuentres que cumplan los requisitos, así como suministrar la información de estos dispositivos en relación a los requisitos.
Usa la información añadida en el contexto para responder a la pregunta pero no hagas referencias al mismo.
Se añade como metadato tanto el proveedor como el nombre a los que pertenece el contexto devuelto.
Responde únicamente con la especificación que se pide. Si no aparece di que no lo sabes.
Pregunta: {question}
Metadato: {metadata}
Contexto: {contexts}
La respuesta debe tener el siguiente formato:
- Nombre del equipo y proveedor.
- Especificaciones que se han preguntado y aparecen en el contexto.
Respuesta:
"""

vectorstore = create_database(f"{path_saved_vector}{name_database}",
                               chunks_texto, nombre_embedding = "text-embedding-3-large")
retriever = create_retriever(vectorstore)
rag = create_RAG(retriever=retriever,
                plantilla_RAG=template_sistema,
                model=modelo_llm)

#### Interacción con el agente

In [4]:
pregunta_ejemplo = """Necesitamos un sensor de medición de ruido con las siguientes características: 
1. Sensor con precisión clase 1 según IEC 61672-1: 
o Detector: Nivel de presión sonora continuo equivalente y nivel de presión sonora con ponderación temporal rápida (Fast) y lenta (Slow). 
o Ponderación Frecuencial: A y C. 
o Funciones Acústicas Medidas: Niveles equivalentes con ponderación frecuencial A y C con tiempo programable entre 1s y 60min: LAeqT y LCeqT. Niveles máximos con ponderación temporal rápida y slow sobre un tiempo programable entre 1s y 60min y ponderación frecuencial A: LAFmaxT y LASmaxT. 
o Resolución 0,1 dB. 
o Precisión según IEC 61672-1: clase 1. 
o Margen de medición sin escalas: de 28 a 120 dBA. 
o Margen de linealidad a 1kHz : de 35 a 120 dBA. 
2. Protección contra agentes externos con kit de exterior: viento, lluvia, pájaros. Mantiene clase 1. Protección IP65. 
3. Capacidad de integración a una plataforma de monitorización de ruido, de código abierto o Propietarias. 
4. Dimensiones reducidas y fácil de instalar en farolas, luminarias, marquesinas, MUPIs, OPIs, vallas y postes publicitarios. 
5. Alimentación de energía, a través la red eléctrica, baterías externas, paneles solares o POE (Power Over Ethernet). 
6. Medición continua 24 horas/7 días a la semana. 
7. Red sin límite en el número de sensores. 
8. Comunicación por Ethernet RJ45, Wi-Fi, Modem. 
9. Configuración remota del sensor, sin necesidad de tener dirigirse hasta el lugar."""

In [13]:
respuesta = rag.invoke(pregunta_ejemplo)

In [14]:
respuesta['answer']

'- Nombre del equipo y proveedor: CESVA Clase 1 modelo TA120, CESVA.\n- Especificaciones que se han preguntado y aparecen en el contexto:\n  - Sensor con precisión clase 1 según IEC 61672-1.\n  - Protección contra agentes externos con kit de exterior: viento, lluvia, pájaros. Mantiene clase 1. Protección IP65.\n  - Completamente integrable en diferentes plataformas: Noise-Platform (CESVA), de código abierto como Sentilo.\n  - Margen de medición sin escalas: de 28 a 120 dBA.\n  - Margen de linealidad a 1kHz: de 35 a 120 dBA.\n  - Red sin límite en el número de sensores.\n  - Comunicación por Ethernet (RJ45), Wi-Fi, Módem.\n  - Configuración remota del sensor.'

#### Interfaz gráfica

In [5]:
import gradio as gr

def predict(message, history):
    response = rag.invoke(message)
    return response['answer']

gr.ChatInterface(predict).launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
